In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

This notebook introduces some basic concepts in tensorflow (TF) and presents the way that I think about my TF models.

Generally, my TF scripts take the following form:

    - Boilerplate (import and preprocess data, etc.)
    - Tensorflow Model (graph)
    - Developing loss functions/training algorithm
    - Training loop
    
The second and third item are where the most TF magic usually happens for me. Lets focus in on the second item the Tensorflow Model (graph).

A model (i.e. ‘computational graph’) is composed on three main parts:

    - Inputs (usually in the form of `placeholder` variables to be filled in later, we'll introduce this in a moment)
    - Tensorflow operations, which are composable functions which, and take `Tensor` objects as inputs and return `Tensor` objects as outputs.
    - Outputs of the model (i.e. network response, etc.). These are actually just more Tensor variables, outputs of operations, but there are usually a few distinguished ones you care about.

Before we get to coding, I just want to make one more point, which is to distinguish `Tensor` objects from `Variables`. When I am working with TF I generally think about two types of tensor object:

    - `Tensors`, which is a general term, but I usualy abuse language by using this to refer specifically to symbolic expressions: TF expressions whose value depends on a yet-to-be-specified input (for example, the output of a network depends on its inputs, which usually are not specified till run-time).
    - `Variables`, which specifically refers to tensor objects with a specific value. Generally these values are stored on the GPU, which is important because it is expensive to load data onto/off of the GPU. A good example of `Variable` objects are network weights. Their values are persistent, and can be used and updated entirely on the GPU, no need to offload them to do gradient updates, etc.
    
Ok. To try and make this distinction clear, lets look at an example. The following code defines an affine transformation (operation with a matrix, plus a vector offset) on a batch of inputs

In [2]:
batch_size = 10
vector_size = 100

input_vectors = tf.placeholder(tf.float32,shape = (batch_size,vector_size))

W = tf.Variable(np.float32(np.random.randn(vector_size,vector_size)))
t = tf.Variable(np.float32(np.random.randn(vector_size)))

output = tf.tensordot(input_vectors,W,axes = [[1],[1]]) + tf.expand_dims(t,0)

mean_out = tf.reduce_mean(output)

Notice that `input_vectors` is a `placeholder`, this means that we can specify its value later on. On the other hand, W and t are `Variable` objects, and I have given the `Variable` function an explicit value (the output of a numpy RNG). This variable object will be stored on the GPU and has a determined value. 

But how do we actually compute the value of `mean_out`? If we just try to print it we get garbage:

In [3]:
print(mean_out)

Tensor("Mean:0", shape=(), dtype=float32)


This is just telling us that `output` is a `Tensor` object. Of course, `mean_out` __can't__ have a value yet, because it depends on `input_vectors`, which doesn't yet have a value!

In order to compute it we need to start a tensorflow `Session`, and tell tensorflow what `input_vectors` is. That looks like this:

In [4]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

data = np.random.randn(batch_size,vector_size)

out = sess.run(mean_out,{input_vectors:data})
print("The average output is: {}".format(out))
sess.close()

The average output is: -0.07564524561166763


The first three lines start a tensorflow `Session`. This is basically spinning up the GPU and getting tensorflow ready to compute stuff. We also create a `global_varialbes_initializer`. The reason for this is that the variables `W` and `t`, while they do have an explicit value, the actual numbers have not yet been plugged into the GPU memory (or something like that). the `global_variables_initializer`, prepares the variables for use by a tensorflow session. Try commenting out the first and third lines from this cell and see what happens!

The next line `data = ...` is just me generating an explicit value that we will feed for `input_vectors`.

The line `out = ...` is the powerhouse here. `sess.run` is the function that performs all the tensorflow computations. The first argument is what we want to compute (here `mean_out`) and the second argument is a `dict` that we can use to feed in values for undetermined tensors. The keys of the `dict` are the tensor objects that need to be specified, and the entries are the value to assign to them. (in the example above, we assign the placeholder `input_vectors` with the value `data`).

Lastly we close the session (we could leave it open and reuse it below, but I wanted to make each cell as independent as possible).

One more example shows that we can specify any tensor in the graph (not just the `input_vectors`).

In [5]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

data = np.random.randn(batch_size,vector_size)

out = sess.run(mean_out,{output:data})
print("The average output is: {}".format(out))
sess.close()

The average output is: -0.04723869264125824


This cell is identical to the last, but we specify the `output` tensor instead of `input_vectors`. Sicne we specify the `output` we no longer need `input_vectors` to compute `mean_out` because once `output` is specified, we can unambiguously determine `mean_out`. 

As one more example, we can specify `W` explicitly if we want.

In [6]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

data = np.random.randn(batch_size,vector_size)
Wtemp1 = np.random.randn(vector_size,vector_size)
Wtemp2 = np.random.randn(vector_size,vector_size)

out = sess.run(mean_out,{input_vectors:data,W:Wtemp1})
print("The average output is: {}".format(out))

out = sess.run(mean_out,{input_vectors:data,W:Wtemp2})
print("The average output is: {}".format(out))

out = sess.run(mean_out,{input_vectors:data})
print("The average output is: {}".format(out))

sess.close()

The average output is: -0.12442449480295181
The average output is: -0.3633689284324646
The average output is: -0.042490653693675995


As you can see in this cell, we specify W to be two different values, and then in the third `sess.run` we leave it unspecified, and it defaults back to the value we initialized it to above. Cool!

So, what you've seen here is a simple example of an input which is acted upon by some variables stored on the GPU, and produces an output. There were four examples of operations given here: the `tensordot` that applies `W` to the input, the `tf.expand_dims` which reshaped `t`, the addition of the two terms to produce `output`, and the `tf.reduce_mean` operation which took the average output. These operations produce tensors as ouput, and computing them with `sess.run` performs that operation. In all the examples above, the operations take symbolic expressions and produce more symbolic expressions, but operations can also do other things, such as change the value of `Variable` objects. 

As an example of this, lets make an operation which shifts `W` by a specified ammount.

In [7]:
change_W = tf.assign(W,W+.01)

the expression `change_W` represents `W` *after* it has been shifted. to actually apply the shift we have to `sess.run` the tensor `change_W`.

In [8]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

w = sess.run(W)
print("shift 0: {}".format(w.mean()))

for k in range(5):
    _ = sess.run(change_W)
    w = sess.run(W)
    print("shift {}: {}".format(k+1,w.mean()))
sess.close()

shift 0: 0.01785106584429741
shift 1: 0.027851058170199394
shift 2: 0.03785105422139168
shift 3: 0.04785104840993881
shift 4: 0.05785103887319565
shift 5: 0.06785102188587189


So, now we've changed the value of `W`. Cool! The thing that confuses me about this sometimes is that the tensor `change_W` contains the instructions to applly the shift. For a long time I thought of the tensor objects as merely representing not-yet-determined values of mathematical expressions, but they are more than that. They contain a set of instructions which can include manipulating variables, combining tensors, and more. 

As a small application, lets use the `tf.assign` function, and gradients, to solve a linear set of equations using gradient descent. 

In [15]:
lr = .001

target_vectors = tf.placeholder(tf.float32,(batch_size,vector_size))
error = tf.reduce_mean((target_vectors - output)**2)
grad = tf.gradients(error,W)
grad_step = tf.assign(W, W - lr * grad[0])

data = np.random.randn(batch_size,vector_size)
target = np.random.randn(batch_size,vector_size)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

err = sess.run(error,{input_vectors:data,target_vectors:target})

print(err)

for k in range(5000):
    w = sess.run(grad_step,{input_vectors:data,target_vectors:target})
    if k%500 == 0:
        err = sess.run(error,{input_vectors:data,target_vectors:target})
        print("Step {} error: {}".format(k,err))
        
sess.close()

125.29
Step 0 error: 125.22870635986328
Step 500 error: 98.48221588134766
Step 1000 error: 77.83393096923828
Step 1500 error: 61.82536315917969
Step 2000 error: 49.35892105102539
Step 2500 error: 39.60639572143555
Step 3000 error: 31.941112518310547
Step 3500 error: 25.88747787475586
Step 4000 error: 21.083398818969727
Step 4500 error: 17.25228500366211


It works! Or at least the error goes down, great. But so slowly!?

What you saw here was defining an error (RMS error between the `output` and a new placeholder`target_vectors`), then computation of the gradient of the error w.r.t. `W`, and then the creation of an operator which takes a single gradient descent step. Then there is the boilerplate of starting a session, and then a "training loop" that perform gradient descent.

Ok, the last thing I want to show you is tensorflows built in tools for optimization, which will work much better than this lame gradient descent. In the previous cell I defined by own gradient descent operation using the `gradients` function. Tensorflow, however, has many algorithms pre-made in the `train` module. We can perform the same optimization using the `Adam` algorithm in the following way:

In [16]:
lr = .001

target_vectors = tf.placeholder(tf.float32,(batch_size,vector_size))
error = tf.reduce_mean((target_vectors - output)**2)

adam = tf.train.AdamOptimizer(learning_rate = lr)
grad_step = adam.minimize(error)

data = np.random.randn(batch_size,vector_size)
target = np.random.randn(batch_size,vector_size)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

err = sess.run(error,{input_vectors:data,target_vectors:target})

print(err)

for k in range(5000):
    w = sess.run(grad_step,{input_vectors:data,target_vectors:target})
    if k%500 == 0:
        err = sess.run(error,{input_vectors:data,target_vectors:target})
        print("Step {} error: {}".format(k,err))
        
sess.close()

97.2818
Step 0 error: 96.75366973876953
Step 500 error: 3.4111878871917725
Step 1000 error: 0.04466744139790535
Step 1500 error: 0.000179307084181346
Step 2000 error: 1.226723185254741e-07
Step 2500 error: 1.035896507461942e-10
Step 3000 error: 2.2060138438195764e-11
Step 3500 error: 7.738922350175592e-12
Step 4000 error: 3.3390271884231604e-12
Step 4500 error: 1.4108542892993237e-12


As you see, adam converged much faster. Although I have trouble taking this advice sometimes: it is usually better to find someone else's implementation than to create your own :P

Great! I hope you have some ideas now of how to do stuff in TF. Check out the other notebooks for full-fledged examples of networks that learn to recognize MNIST digits. 

Some stuff for you to try here is:

    - Change the error function and see how it effects learning (for example, what about using `tf.reduce_max` instead of `tf.reduce_mean`?)
    - Modify the code so that it perform linear regression on the batch, instaed of just solving a linear regression problem, by specifying the "true" transofrmation and noise, generating batches of data, and fitting `W` and `t`.